# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 

In [102]:
%reset -f

# Download dataset

In [103]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip ./gdrive/MyDrive/Colab\ Notebooks/HW4/Dataset.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# CONFIG

In [104]:
import torch
import numpy as np

CONFIG = {
    "MODEL_PATH": "gdrive/MyDrive/Colab Notebooks/HW4/models/model_conformer",
    "BEST_VAL_PATH": "gdrive/MyDrive/Colab Notebooks/HW4/models/best_val_conformer",  
    'SEGMENT_LEN': 128,
}

# set random seed for reproducibility
SEED = 12345
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [105]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=CONFIG['SEGMENT_LEN']):
    self.data_dir = data_dir
    self.segment_len = segment_len
 
    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
 
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]
 
    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
  def __len__(self):
    return len(self.data)
 
  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))
 
    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
          
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker
 
  def get_speaker_number(self):
    return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [106]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True)
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [107]:
# reference: https://github.com/sooftware/conformer
import sys
sys.path.append('gdrive/MyDrive/Colab Notebooks/HW4/conformer')
from conformer.encoder import ConformerEncoder

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, d_model=256, n_spks=600, dropout=0.1):
    super().__init__()
    segment_len = CONFIG['SEGMENT_LEN']
    input_dim = 40

    # Normalization
    self.bn0 = nn.BatchNorm1d(input_dim, momentum=0.1)

    self.encoder = ConformerEncoder(input_dim=input_dim, encoder_dim=d_model, conv_kernel_size=31, num_layers=1)

    self.linear1 = nn.Linear(d_model, 1024)
    self.bn1 = nn.BatchNorm1d(1024, momentum=0.1)

    self.out = nn.Linear(1024, n_spks)

    # activation function and dropout
    self.act_fn = nn.ReLU(inplace=True)
    self.dropout = nn.Dropout(p=0.5)

  def forward(self, inputs):
    """
    B: batch_size, S: segment_length, F: feature_num
    I: input_dim(40), O: output_dim(n_spks)

    bn: expect shape of (B, F, S)
    linear: expect shape of (B, *, F)
    encoder: expect shape of (B, S, F)
    conv: expect shape of (B, F, S)

    args:
      inputs: (B, S, F/I)
    return:
      outs: (B, F/O)
    """
    # normalize input
    inputs = inputs.transpose(1, 2) # (B, F, S)
    inputs = self.bn0(inputs) # (B, F, S)
    inputs = inputs.transpose(1, 2) # (B, S, F)

    # transformer
    inputs, _ = self.encoder(inputs, torch.LongTensor(inputs.shape[0])) # (B, S, F)

    # get mean of feature
    inputs = inputs.mean(dim=1) # (B, F)

    # ff_layer
    inputs = self.linear1(inputs) # (B, F)
    inputs = self.bn1(inputs) # (B, F)
    inputs = self.act_fn(inputs)
    inputs = self.dropout(inputs)

    # output
    outputs = self.out(inputs) # (B, F)

    return outputs


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [109]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR

def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [110]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [111]:
from tqdm import tqdm
import torch

def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr", position=0, leave=True)

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.5f}",
      accuracy=f"{running_accuracy / (i+1):.5f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


# Main function (Training)

In [ ]:
import os
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
      "model_path": CONFIG['MODEL_PATH'],
    "best_val_path": CONFIG['BEST_VAL_PATH'],
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 0,
    "save_steps": 2000,
    "total_steps": 200000,
  }

  return config


def main(
  data_dir,
  model_path,
  best_val_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  if os.path.isfile(model_path):
    print('loading previous model parameters...')
    ckpt = torch.load(model_path, map_location='cpu')  # Load your best model
    model.load_state_dict(ckpt)

  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters())
  # scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None
  if os.path.isfile(best_val_path):
    with open(best_val_path, 'r') as f:
      best_accuracy = float(f.read())
      print(f'best_accuracy: {best_accuracy}')

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step", position=0, leave=True)

  tot_batch_loss = 0
  tot_batch_accuracy = 0
  tot_step = 0
  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()
    tot_batch_loss += batch_loss
    tot_batch_accuracy += batch_accuracy

    # Updata model
    loss.backward()
    optimizer.step()
    # scheduler.step()
    optimizer.zero_grad()
    
    # Log
    tot_step += 1
    pbar.update()
    pbar.set_postfix(
      loss=f"{tot_batch_loss/tot_step:.5f}",
      accuracy=f"{tot_batch_accuracy/tot_step:.5f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      tot_step = 0
      tot_batch_loss = 0
      tot_batch_accuracy = 0
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step", position=0, leave=True)

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, model_path)
      with open(best_val_path, 'w') as f:
        f.write(str(best_accuracy))

      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [03:16<00:00, 10.20 step/s, accuracy=0.20425, loss=4.16756, step=2000]
Valid: 100% 6944/6944 [00:27<00:00, 250.42 uttr/s, accuracy=0.38465, loss=2.80796]
Train:   0% 4/2000 [00:00<12:35,  2.64 step/s, accuracy=0.31250, loss=3.06093, step=2004]

Step 2000, best model saved. (accuracy=0.3846)


...

Train: 100% 2000/2000 [04:04<00:00,  8.20 step/s, accuracy=0.90075, loss=0.34565, step=158000]
Valid: 100% 6944/6944 [00:30<00:00, 230.49 uttr/s, accuracy=0.90639, loss=0.40398]
Train:   0% 4/2000 [00:01<28:16,  1.18 step/s, accuracy=0.89062, loss=0.37750, step=158004]

Step 158000, best model saved. (accuracy=0.9083)


Train: 100% 2000/2000 [04:06<00:00,  8.10 step/s, accuracy=0.89942, loss=0.35337, step=160000]
Valid: 100% 6944/6944 [00:29<00:00, 233.31 uttr/s, accuracy=0.90495, loss=0.41157]
Train:   0% 4/2000 [00:00<06:44,  4.94 step/s, accuracy=0.89844, loss=0.46231, step=160004]

Step 160000, best model saved. (accuracy=0.9083)


Train:  92% 1846/2000 [03:47<00:22,  6.74 step/s, accuracy=0.90037, loss=0.35240, step=161845]

# Inference (Testing)

## Dataset of inference (Testing)

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset

class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]
    self.segment_len = CONFIG['SEGMENT_LEN']

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    if len(mel) < self.segment_len:
      mel_new = torch.zeros(self.segment_len, 40)
      mel_new[:mel.shape[0], :40] = mel
      mel = mel_new
      mel = torch.FloatTensor(mel)

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)



## Main funcrion of Inference (Testing)

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "model_path": CONFIG['MODEL_PATH'],
    "output_path": "./output.csv",
  }

  return config

def main(
  data_dir,
  model_path,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.zeros = model.zeros.to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())
